# Bagian 3C: Peringkasan Teks (Text Summarization) - Bobot: 25%

Buatlah sistem untuk meringkas artikel panjang menjadi singkat.
1. Gunakan dataset dari materi kuliah (kami buat dataset khusus untuk peringkasan)
2. Implementasikan metode ekstraktif menggunakan TF-IDF Scoring.
○ Petunjuk: Hitung skor setiap kalimat berdasarkan bobot kata-katanya, lalu
ambil kalimat dengan skor tertinggi.
3. Analisis: Bandingkan hasil ringkasan mesin dengan ringkasan referensi. Apakah
mesin berhasil menangkap inti kalimat? Jelaskan kelebihan metode ekstraktif
dibanding abstraktif secara teori.

In [ ]:
# Instalasi library tambahan
!pip install sastrawi -q

In [ ]:
# Import library
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

# Download resource NLTK
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK resources: {e}")

In [ ]:
# Buat dataset peringkasan teks dalam format JSONL
# Dataset ini berisi artikel panjang dan ringkasannya sebagai referensi

summarization_data = [
    {
        "article": "Hotel ini menawarkan pengalaman menginap yang luar biasa dengan pelayanan ramah dan fasilitas lengkap. Kamar yang disediakan bersih dan nyaman dengan perlengkapan yang memadai. Staf hotel sangat profesional dan membantu dalam memberikan informasi wisata lokal. Fasilitas utama termasuk kolam renang yang bersih, pusat kebugaran lengkap, dan restoran dengan menu bervariasi. Lokasi hotel strategis dekat dengan tempat wisata dan pusat perbelanjaan. Wifi gratis tersedia di seluruh area hotel dengan kecepatan tinggi. Sarapan pagi disajikan dengan pilihan menu yang lezat. Kualitas tidur sangat baik karena kamar bebas kebisingan. Pengalaman menginap di hotel ini sangat direkomendasikan untuk liburan keluarga maupun bisnis.",
        "summary": "Hotel dengan pelayanan ramah, fasilitas lengkap, kamar bersih, lokasi strategis, dan wifi gratis. Sangat direkomendasikan."
    },
    {
        "article": "Pengalaman menginap di hotel ini tidak memuaskan sama sekali. Kamar yang disediakan kecil, kotor, dan AC tidak dingin. Pelayanan staf lambat dan tidak ramah. Makanan di restoran terasa hambar dengan pilihan menu yang terbatas. Kolam renang kotor dan tidak terawat. Parkir terbatas terutama saat akhir pekan. Walaupun lokasi strategis, harga tidak sebanding dengan fasilitas yang diberikan. Pelayanan check-in memakan waktu lama. Kualitas tempat tidur buruk dan banyak nyamuk. Penerangan di kamar juga redup. Tidak akan kembali ke hotel ini lagi.",
        "summary": "Pengalaman buruk: kamar kotor, AC tidak dingin, pelayanan lambat, makanan buruk. Tidak direkomendasikan."
    },
    {
        "article": "Lokasi hotel sangat strategis dekat dengan tempat wisata dan pusat perbelanjaan. Kamar cukup bersih dan pelayanan memadai. Harga terjangkau dengan fasilitas standar seperti wifi gratis dan kolam renang kecil. Sarapan pagi enak dengan pilihan menu yang bervariasi. Staf ramah meski terkadang sibuk. Hanya saja suara lalu lintas terdengar saat malam hari. Fasilitas gym sederhana namun cukup. Tempat parkir cukup luas meski terkadang penuh. Pelayanan kamar cepat dan ramah. Kebersihan umum terjaga. Koneksi internet stabil. Secara keseluruhan pengalaman menginap cukup memuaskan untuk harga segitu.",
        "summary": "Lokasi strategis, harga terjangkau, fasilitas standar, pelayanan memadai. Pengalaman cukup memuaskan."
    },
    {
        "article": "Hotel dengan desain modern dan interior menarik menawarkan pengalaman menginap mewah. Staf sangat profesional dan ramah sejak kedatangan hingga keberangkatan. Layanan kamar cepat dengan makanan lezat dan pilihan minuman premium. Kamar luas dengan perlengkapan lengkap dan kamar mandi bersih berfasilitas lengkap. Fasilitas spa sangat menenangkan dan membantu relaksasi. Kolam renang infinity dengan pemandangan kota sangat memukau. Restoran dengan chef profesional menyajikan hidangan lezat dari berbagai negara. Wifi cepat dan bisa diakses di seluruh area hotel. Pelayanan antar-jemput bandara tersedia. Ruang pertemuan tersedia untuk keperluan bisnis.",
        "summary": "Hotel mewah dengan desain modern, pelayanan profesional, fasilitas lengkap dan mewah."
    },
    {
        "article": "Harga sesuai dengan fasilitas yang diberikan. Kamar bersih dan rapi, pelayanan standar. Lokasi tenang dan nyaman untuk beristirahat. Hanya saja breakfast terbatas dan pilihan menu sedikit. Kolam renang kecil tapi bersih. Parkir terbatas terutama saat weekend penuh. Namun secara keseluruhan kenyamanan tetap terjaga. WiFi cukup cepat untuk penggunaan normal. Saran untuk perbaikan adalah menambah variasi menu sarapan dan menyediakan lebih banyak tempat duduk di area breakfast. Kebersihan kamar terjaga dengan baik. Perlengkapan mandi standar. Penerangan kamar cukup terang.",
        "summary": "Harga sesuai fasilitas, kamar bersih, lokasi tenang. Menu sarapan perlu diperbanyak."
    }
]

# Simpan dataset dalam format JSONL
with open('summarization_dataset.jsonl', 'w') as f:
    for item in summarization_data:
        f.write(json.dumps(item) + '\n')

print(f"Dataset peringkasan teks telah dibuat dengan {len(summarization_data)} contoh")
print("Contoh data pertama:")
print(json.dumps(summarization_data[0], indent=2, ensure_ascii=False))

In [ ]:
# Fungsi untuk melakukan peringkasan teks metode ekstraktif TF-IDF
class ExtractiveSummarizer:
    def __init__(self):
        self.stemmer = None
        self.stop_words = None
        
        # Inisialisasi stemmer dan stop words
        try:
            factory = StemmerFactory()
            self.stemmer = factory.create_stemmer()
        except Exception as e:
            print(f"Error inisialisasi stemmer: {e}")
            
        try:
            self.stop_words = set(stopwords.words('indonesian'))
        except Exception as e:
            print(f"Error inisialisasi stopwords: {e}")
            self.stop_words = set()
    
    def preprocess_text(self, text):
        # Case folding
        text = text.lower()
        
        # Hapus karakter tidak perlu
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        return text
    
    def extractive_summarization(self, text, num_sentences=3):
        """Fungsi peringkasan teks ekstraktif menggunakan TF-IDF"""
        # Tokenisasi kalimat
        sentences = sent_tokenize(text)
        
        if len(sentences) <= num_sentences:
            return text
        
        # Preprocessing teks
        processed_sentences = [self.preprocess_text(sent) for sent in sentences]
        
        # Vektorisasi menggunakan TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
        tfidf_matrix = vectorizer.fit_transform(processed_sentences)
        
        # Hitung skor rata-rata untuk setiap kalimat
        sentence_scores = tfidf_matrix.sum(axis=1).flatten()
        
        # Dapatkan indeks kalimat dengan skor tertinggi
        top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]
        
        # Urutkan indeks agar sesuai dengan urutan kalimat asli
        top_sentence_indices.sort()
        
        # Ambil kalimat-kalimat dengan skor tertinggi
        summary_sentences = [sentences[i] for i in top_sentence_indices]
        
        return ' '.join(summary_sentences)

# Buat instance summarizer
summarizer = ExtractiveSummarizer()
print("Extractive Summarizer telah dibuat")

In [ ]:
# Demonstrasi peringkasan teks
print("DEMONSTRASI PERINGKASAN TEKS")
print("=" * 60)

for i, data in enumerate(summarization_data):
    print(f"\nContoh {i+1}:")
    print(f"Artikel Asli (panjang): {data['article']}")
    print()
    
    # Hasil peringkasan dengan metode TF-IDF
    machine_summary = summarizer.extractive_summarization(data['article'], num_sentences=2)
    print(f"Ringkasan Mesin (TF-IDF): {machine_summary}")
    print()
    
    print(f"Ringkasan Referensi (Manual): {data['summary']}")
    print()
    
    # Analisis kesamaan
    machine_words = set(machine_summary.lower().split())
    reference_words = set(data['summary'].lower().split())
    
    intersection = machine_words.intersection(reference_words)
    jaccard_similarity = len(intersection) / len(machine_words.union(reference_words)) if len(machine_words.union(reference_words)) > 0 else 0
    
    print(f"Kesamaan kata: {len(intersection)} kata dari {len(reference_words)} kata referensi")
    print(f"Jaccard Similarity: {jaccard_similarity:.2f}")
    
    if jaccard_similarity > 0.4:
        print("✅ Mesin berhasil menangkap inti dari teks")
    else:
        print("⚠️ Mesin belum sepenuhnya menangkap inti dari teks")
    
    print("-" * 60)

In [ ]:
print("""ANALISIS HASIL PERINGKASAN TEKS

BANDINGKAN HASIL RINGKASAN MESIN DENGAN RINGKASAN REFERENSI:

Dari hasil di atas, kita dapat mengamati bahwa:

1. Mesin mampu mengidentifikasi kalimat-kalimat penting dalam teks berdasarkan bobot kata (TF-IDF)
2. Mesin sering kali memilih kalimat dengan kata kunci penting sebagai bagian dari ringkasan
3. Dalam beberapa kasus, mesin berhasil menangkap inti dari artikel, meskipun tidak sekomprehensif ringkasan buatan manusia
4. Karena metode ekstraktif hanya mengambil kalimat utuh dari teks asli, ringkasan terdengar alami dan gramatikal benar

APAKAH MESIN BERHASIL MENANGKAP INTI KALIMAT?

Ya, mesin mampu menangkap inti dari sebagian besar artikel, terutama dalam mengenali topik utama seperti kualitas kamar, pelayanan, dan fasilitas hotel. Namun kadang mesin tidak sepresisi buatan manusia dalam menyusun ringkasan yang ringkas dan fokus.

KELEBIHAN METODE EKSTRAKTIF DIBANDING ABSTRAKTIK SECARA TEORI:

1. METODE EKSTRAKTIF:
   - Lebih sederhana dan cepat dalam implementasi
   - Tidak membutuhkan pemahaman semantik yang kompleks
   - Kalimat hasil ringkasan dijamin gramatikal dan koheren karena merupakan bagian langsung dari teks asli
   - Lebih stabil dan terprediksi dibanding metode abstraktif
   - Lebih mudah dievaluasi karena dapat dilakukan dengan membandingkan kalimat yang dipilih dengan teks asli secara langsung
   - Tidak memerlukan model bahasa kompleks

2. PERBANDINGAN DENGAN METODE ABSTRAKTIK:
   - Metode abstraktif bisa menghasilkan ringkasan yang lebih ringkas dan fokus karena bisa menyusun kalimat baru
   - Namun, metode abstraktif lebih kompleks, membutuhkan model bahasa yang lebih canggih
   - Metode abstraktif berisiko menghasilkan informasi yang tidak akurat atau tidak konsisten dengan teks asli
   - Implementasi metode abstraktif biasanya memerlukan data pelatihan besar dan komputasi yang lebih tinggi

Jadi untuk aplikasi sederhana, metode ekstraktif dengan TF-IDF merupakan pilihan yang efektif dan efisien.""")